In [1]:
import pandas as pd
import numpy as np
import math
import operator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 
fruit= pd.read_table("C:/Users/Simon/Downloads/Machine Learning/fruit_data_with_colors.txt")
fruit.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [2]:
X=fruit.iloc[:,3:].values

y=fruit.iloc[:,1].values
print(X[0:5])
print(y[0:5])

[[192.     8.4    7.3    0.55]
 [180.     8.     6.8    0.59]
 [176.     7.4    7.2    0.6 ]
 [ 86.     6.2    4.7    0.8 ]
 [ 84.     6.     4.6    0.79]]
['apple' 'apple' 'apple' 'mandarin' 'mandarin']


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

print(X_train[0:5])

[[ 0.60031901  0.4257231   1.03348893  0.09030379]
 [-0.36363971 -0.36766995 -0.21277713 -0.17415731]
 [ 0.79311076  0.19903937  1.72585897 -0.7030795 ]
 [ 0.31113139  0.08569751  1.03348893 -0.57084895]
 [-1.39186235 -1.38774674 -2.35912425  0.61922598]]


In [4]:
for x in range (1,9):
    classifier = KNeighborsClassifier(n_neighbors=x)  
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print("n_neighbors=",x)
    print(confusion_matrix(y_test, y_pred))
    print("kNN %):", metrics.accuracy_score(y_test, y_pred)*100)
    print()

n_neighbors= 1
[[8 0 0 0]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 94.44444444444444

n_neighbors= 2
[[8 0 0 0]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 94.44444444444444

n_neighbors= 3
[[7 0 0 1]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 88.88888888888889

n_neighbors= 4
[[8 0 0 0]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 94.44444444444444

n_neighbors= 5
[[5 0 0 3]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 77.77777777777779

n_neighbors= 6
[[5 0 0 3]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 77.77777777777779

n_neighbors= 7
[[5 0 0 3]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 77.77777777777779

n_neighbors= 8
[[5 0 0 3]
 [0 5 0 0]
 [0 0 1 0]
 [1 0 0 3]]
kNN %): 77.77777777777779



In [5]:
gnb = GaussianNB()
y_pred2 = gnb.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred2))
print("gnb %):", metrics.accuracy_score(y_test, y_pred2)*100)

[[8 0 0 0]
 [0 5 0 0]
 [0 0 1 0]
 [2 0 0 2]]
gnb %): 88.88888888888889


In [6]:
class gnbImp(object):
    
    def fit(self,X,y):
        self.classes = np.unique(y)
        print(self.classes)
        classes,count = np.unique(y, return_counts=True)
        self.freqOfClass = dict(zip(classes,count))
        indOfClassData={}
        featDataOfClass={}
        total = sum(self.freqOfClass.values())
        for cls in classes:
            self.freqOfClass[cls]= self.freqOfClass[cls]/total
    #         gets indices of data that is under the respective class
            indOfClassData[cls] = np.argwhere(y==cls)
    #         uses the indices to obtain the data of the features of each class
            featDataOfClass[cls] = X[indOfClassData[cls],:]

        self.means={}
        self.stds={}

        for cls in self.classes:
             self.means[cls]= np.mean(featDataOfClass[cls], axis=0)[0]
             self.stds[cls] = np.std(featDataOfClass[cls], axis=0)[0]
            
        #calculate probabilities of the set of data being in a class
    def calcProb(self,x,mean,std):
        #formula (3)
        if (std==0):
            prob=0
        else:
            prob = (1 / (math.sqrt(2 * math.pi) * std)) * math.exp(-((x - mean) ** 2 / (2 * std ** 2)))
        return prob
    
        #uses calcProb to calculate the probabilities of the set of data to be in the different classes
    def predictProb(self,X):
        #P(c)
        self.classProb = {cls:self.freqOfClass[cls] for cls in self.classes}
        for cls in self.classes:
            for j in range(len(self.means)):
                #P(d|c)*P(c)
                #P(d|c)=P(x1|c)P(x2|c)P(x3|c)…
                self.classProb[cls]=self.calcProb(X[j],self.means[cls][j],self.stds[cls][j])*self.classProb[cls]
        return self.classProb
    
    def predict(self,X):
        #finalized predictions in this array
        predictions = []
        for x in X:#go through each row
            predictedClass = None
            for predProb in self.predictProb(x).items():
        #takes the class with the highest probability
                print(self.predictProb(x).items())
                predictedClass = max(self.predictProb(x).items(), key=operator.itemgetter(1))[0]
            predictions.append(predictedClass)
        return predictions

In [7]:
gnbImp2 = gnbImp()
gnbImp2.fit(X_train, y_train)
y_pred3 = gnbImp2.predict(X_test)
print(confusion_matrix(y_test, y_pred3))
print("gnb %):", metrics.accuracy_score(y_test, y_pred3)*100)

['apple' 'lemon' 'mandarin' 'orange']
dict_items([('apple', 0.044862389824926886), ('lemon', 1.4095741745691487e-33), ('mandarin', 0.0), ('orange', 3.607096684622342e-10)])
dict_items([('apple', 0.044862389824926886), ('lemon', 1.4095741745691487e-33), ('mandarin', 0.0), ('orange', 3.607096684622342e-10)])
dict_items([('apple', 0.044862389824926886), ('lemon', 1.4095741745691487e-33), ('mandarin', 0.0), ('orange', 3.607096684622342e-10)])
dict_items([('apple', 0.044862389824926886), ('lemon', 1.4095741745691487e-33), ('mandarin', 0.0), ('orange', 3.607096684622342e-10)])
dict_items([('apple', 0.08116908430712151), ('lemon', 2.6610750067640133e-13), ('mandarin', 1.073052199788044e-255), ('orange', 0.013871982081876835)])
dict_items([('apple', 0.08116908430712151), ('lemon', 2.6610750067640133e-13), ('mandarin', 1.073052199788044e-255), ('orange', 0.013871982081876835)])
dict_items([('apple', 0.08116908430712151), ('lemon', 2.6610750067640133e-13), ('mandarin', 1.073052199788044e-255), (